In [1]:
pip install tensorflow pandas numpy matplotlib scikit-learn


   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.8/385.2 MB 2.1 MB/s eta 0:03:04
   ---------------------------------------- 1.3/385.2 MB 2.3 MB/s eta 0:02:46
   ---------------------------------------- 1.8/385.2 MB 2.3 MB/s eta 0:02:44
   ---------------------------------------- 2.4/385.2 MB 2.2 MB/s eta 0:02:52
   ---------------------------------------- 2.6/385.2 MB 2.2 MB/s eta 0:02:53
   ---------------------------------------- 3.1/385.2 MB 2.2 MB/s eta 0:02:54
   ---------------------------------------- 3.7/385.2 MB 2.2 MB/s eta 0:02:52
   ---------------------------------------- 4.2/385.2 MB 2.2 MB/s eta 0:02:53
   ---------------------------------------- 4.7/385.2 MB 2.2 MB/s eta 0:02:51
    --------------------------------------- 5.0/385.2 MB 2.2 MB/s eta 0:02:54
    --------------------------------------- 5.5/385.2 MB 2.2 MB/s eta 0:02:55


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


In [3]:
pip show tensorflow


Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\Users\THARUKA\anaconda3\envs\Research_Project_env\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.
